# 변동성 돌파 전략 in 파이썬을 이용한 비트코인 자동매매
## reference: https://wikidocs.net/21891

#### 1) 가격 변동폭 계산: 투자하려는 가상화폐의 전일 고가(high)에서 전일 저가(low)를 빼서 가상화폐의 가격 변동폭을 구합니다.
#### 2) 매수 기준: 당일 시간에서 (변동폭 * ratio) 이상 상승하면 해당 가격에 바로 매수합니다.
#### 3) 매도 기준: 당일 종가에 매도합니다.
#### + (reference 와의 차이점)
#### 4) 가상화폐는 24시간 거래됨으로 아래에서는 freq 변수를 이용하여 하루의 기준을 정합니다.
#### 5) ccxt 패키지를 사용해서 Binance 거래소 데이터 사용

In [1]:
from datetime import datetime, timezone
import ccxt
import time
import pandas as pd

# Market: Binance
binance = ccxt.binance({'enableRateLimit': True})

In [2]:
# Hyperparameters
ticker = 'XRP/USDC'
freq = 24 # hours
original_seed = 50 # seed money, usdc
ratio = 0.5
days = 10 # historical data for back-testing. historical data: (current_moment - n_days) ~ current_moment

# Variables
amount = 0 # amount of crypto to buy/sell
previous_time = 0
target_price = 0
buy_flag = 1
taget_flag = 1 
ror_list = list()

In [3]:
# Get historical data
days_in_ms = days * 24 * 60 * 60 * 1000 # millisecond
now = datetime.now().timestamp()
now_in_ms = (now - now % 60) * 1000
since = now_in_ms - days_in_ms

ohlcvs = list()
for since_ in range(int(since), int(now_in_ms), 1000*1000*60):
    ohlcvs.extend(binance.fetch_ohlcv(ticker, since=since_, limit=1000))
    time.sleep(binance.rateLimit / 1000)

In [4]:
# Concatenate o, h, l, c
# Useless
price_history = list()
for item in ohlcvs:
    price_history.extend(item[1:5])

In [5]:
def get_target_price(ohlcvs, freq, timestep, ratio):
    df = pd.DataFrame(ohlcvs[timestep - freq * 60:timestep])
    df.columns =  ['time', 'open', 'high', 'low', 'close', 'volume']
    df = df.set_index('time')
    
    current_open  = ohlcvs[timestep][1] # open price
    previous_high = max(df.high)
    previous_low = max(df.low)
    target_price = current_open + (previous_high - previous_low) * ratio
    return target_price

In [6]:
seed = original_seed

for timestep, ohlcv in enumerate(ohlcvs):
    current_time = timestep
    if_sell = timestep - previous_time >=  60 * freq
    
    if current_time % 60  == 0 and if_sell:
        #from IPython.core.debugger import Tracer; Tracer()() 
        if amount > 0:
            previous_seed = seed
            seed = ohlcv[1] * amount * 0.999 # use lowest price,transaction fee: 0.1%
            ror_list.append(1 + (seed - previous_seed) / previous_seed)
            print('Sell at:\t%f'%ohlcv[1])
            print('------------------------')
            buy_flag = 1
        
        target_price = get_target_price(ohlcvs, freq, current_time, ratio)
        print('Target Price:\t%f'%target_price)
        target_flag = 0
        
        previous_time = current_time

    
    # Get current price
    # 틱단위 데이터 혹은 초 단위 데이터가 존재 하지 않음으로 open, high, low, close 가격을 순차적 가격으로 가정합니다.
    for price in ohlcv[1:5]:
        # Compare target price with the current price
        if price > target_price and buy_flag == 1 and target_price != 0:
            print('Buy at:\t\t%f'%price)
            amount = seed * 0.999 / price # transaction fee: 0.1%
            buy_flag = 0

print('\nOriginal seed money:\t%f'%original_seed)
print('Final seed money:\t%f'%seed)    

# Approximately Calculate MDD (Maximum Draw Down)
# MDD(Maximum Draw Down)은 투자 기간 중에 포트폴리오의 전 고점에서 저점까지의 최대 누적 손실을 의미합니다. 
# MDD를 계산하는 수식은 다음과 같습니다. 
# max는 전고점 low는 저점을 의미합니다.
# MDD = (high - low) /  high * 100
# ror: rate of return
# hpr: holding peroid return
df = pd.DataFrame({'ror': ror_list})
df['hpr'] = df['ror'].cumprod()
df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax() * 100
print('MDD(%%): %f'%(df['dd'].max()))

Target Price:	0.217220
Buy at:		0.217670
Sell at:	0.235220
------------------------
Target Price:	0.236270
Buy at:		0.236660
Sell at:	0.226160
------------------------
Target Price:	0.226785
Buy at:		0.228300
Sell at:	0.218770
------------------------
Target Price:	0.218815
Buy at:		0.219130
Sell at:	0.229710
------------------------
Target Price:	0.231860
Buy at:		0.231990
Sell at:	0.235180
------------------------
Target Price:	0.235550
Buy at:		0.235710
Sell at:	0.226300
------------------------
Target Price:	0.226800
Buy at:		0.227070
Sell at:	0.297170
------------------------
Target Price:	0.299050
Buy at:		0.300170
Sell at:	0.322960
------------------------
Target Price:	0.325370
Buy at:		0.325450
Sell at:	0.319170
------------------------
Target Price:	0.320055

Original seed money:	50.000000
Final seed money:	68.466389
MDD(%): 8.791620
